In [1]:
# pip install -r requirements.txt

import os
from pyspark.sql import SparkSession

jar_dir = "/home/bnguyen/Desktop/DE_project/notebooks/jars"
os.environ["PYSPARK_SUBMIT_ARGS"] = (
    f"--jars {jar_dir}/hadoop-azure-3.3.6.jar,"
    f"{jar_dir}/azure-storage-8.6.6.jar,"
    f"{jar_dir}/jetty-client-9.4.43.v20210629.jar,"
    f"{jar_dir}/jetty-http-9.4.43.v20210629.jar,"
    f"{jar_dir}/jetty-io-9.4.43.v20210629.jar,"
    f"{jar_dir}/jetty-util-9.4.43.v20210629.jar,"
    f"{jar_dir}/jetty-util-ajax-9.4.43.v20210629.jar "
    "pyspark-shell"
)


In [2]:
# Init spark session
spark = SparkSession.builder \
    .appName("Bronze to Silver: Table filter") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/09 13:18:30 WARN Utils: Your hostname, lenovo-slim, resolves to a loopback address: 127.0.1.1; using 192.168.199.13 instead (on interface wlp2s0)
25/07/09 13:18:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


25/07/09 13:18:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/07/09 13:18:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/07/09 13:18:41 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/07/09 13:18:41 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
# Bronze access key
spark.conf.set(
    "fs.azure.account.key.mybronze.blob.core.windows.net",
    "c5etqTidViezB/4ukOAALy23HeMBsJJ8g+2nFaIdbC7E9PhLw0y2YIA1ItjutpqS1/8Ga8fw40mR+ASt2T+/sw=="
)

# Silver access key
spark.conf.set(
    "fs.azure.account.key.mysilver.dfs.core.windows.net",
    "bAthp0pVBfqEtyCvJElSX7MeI7ejSLa6cjuPoMz0Gg/69uzEW01y4URMDXsdFCrkpc9M54cDHnXs+AStj1gExQ=="
)

In [4]:
# Define storage account 
storage_account_bronze = "mybronze"
bronze_container = "bronze"
storage_account_silver = "mysilver"
silver_container = "silver"

tables = ["Customers", "Products", 
          "Sellers", "Orders", 
          "OrderItems","ProductCategories",
          "OrderStatus","Reasons", "Payments"]


In [5]:
for table in tables:
    bronze_path = f"wasbs://{bronze_container}@{storage_account_bronze}.blob.core.windows.net/{table}"
    silver_path = f"abfss://{silver_container}@{storage_account_silver}.dfs.core.windows.net/{table}"

    print(f"Processing {table}...")
    
    df = spark.read.parquet(bronze_path)
    df.write.mode("overwrite").parquet(silver_path)

    print(f"Written to {silver_path}")

    
    

Processing Customers...


25/07/09 13:18:43 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-azure-file-system.properties,hadoop-metrics2.properties


Written to abfss://silver@mysilver.dfs.core.windows.net/Customers
Processing Products...


Written to abfss://silver@mysilver.dfs.core.windows.net/Products
Processing Sellers...


Written to abfss://silver@mysilver.dfs.core.windows.net/Sellers
Processing Orders...


...

Written to abfss://silver@mysilver.dfs.core.windows.net/OrderItems
Processing ProductCategories...


Written to abfss://silver@mysilver.dfs.core.windows.net/ProductCategories
Processing OrderStatus...


Written to abfss://silver@mysilver.dfs.core.windows.net/OrderStatus
Processing Reasons...


Written to abfss://silver@mysilver.dfs.core.windows.net/Reasons
Processing Payments...


Written to abfss://silver@mysilver.dfs.core.windows.net/Payments
